In [1]:
from model_train.generate_filters11 import Kernel as Kernel24
from model_train.generate_filter32 import Kernel as Kernel32
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms, datasets, utils

## components

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):   
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(  
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55]
            nn.ReLU(inplace=True), #inplace
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27]
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [3]:
def test_model(net,test_loader):
    acc = 0.0
    test_acc = 0.0
    net.eval()  
    with torch.no_grad():
        for test_data in test_loader:
            test_images, test_labels = test_data
            y_pred_prob = net(test_images.to(device))
            y_pred = torch.max(y_pred_prob, dim=1)[1]
            test_acc += torch.sum(y_pred==test_labels.to(device)).item()/len(test_labels)
    acc = test_acc/len(test_loader)
    return acc

## data loder

In [4]:
#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# preparing data
data_transform = {
"train": transforms.Compose([transforms.RandomResizedCrop(224),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
"val": transforms.Compose([transforms.Resize((224, 224)),   # cannot 224, must (224, 224)
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
"test": transforms.Compose([transforms.Resize((224, 224)),   # cannot 224, must (224, 224)
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                }

batch_size = 8
train_dataset = datasets.ImageFolder(root="../data/train", transform=data_transform["train"])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

validate_dataset = datasets.ImageFolder(root="../data/val", transform=data_transform["val"])
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

test_dataset = datasets.ImageFolder(root="../data/test", transform=data_transform["test"])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    
class_num = 90


cuda:0


## model load

In [5]:
model = AlexNet(num_classes=class_num, init_weights=False)
save_path = "params/AlexNet.pth"
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()
print('Test accuracy for original = ',test_model(model,test_loader))

Test accuracy for original =  0.3628691983122363


In [6]:
model2 = models.alexnet(pretrained=True)
save_path = "params/pre_trained_AlexNet.pth"
model2.load_state_dict(torch.load(save_path))
model2.to(device)
model2.eval()
print('Test accuracy for original = ',test_model(model2,test_loader))

Test accuracy for original =  0.5411392405063291


## substitute params

In [7]:
def rand_weight(k,seed=0):
    #print(k)
    np.random.seed(seed)
    change_r = np.random.randint(2)
    change_g = np.random.randint(2)
    change_b = np.random.randint(2)
    weight = np.zeros((3,11,11))
    weight[0] = -k if change_r else k
    weight[1] = -k if change_g else k
    weight[2] = -k if change_b else k
    return weight

In [8]:
kernels = Kernel24()()
kernels_inv = kernels * -1
full_kernels = torch.vstack((kernels,kernels_inv))
print(full_kernels.shape)

torch.Size([48, 11, 11])


In [9]:
weights0 = [rand_weight(k) for k in full_kernels]
weights0 = np.array(weights0)
weights0 = torch.tensor(weights0)
weights0 = weights0.to(torch.float32)
print(weights0.dtype)
print(model.features[0].weight.shape)
weights0 = nn.Parameter(weights0)
model.features[0].weight = weights0

torch.float32
torch.Size([48, 3, 11, 11])


In [10]:
model.to(device)
model.eval()
print('Test accuracy for substitute = ',test_model(model,test_loader))

Test accuracy for substitute =  0.01740506329113924


In [11]:
kernels = Kernel32()()
kernels_inv = kernels * -1
full_kernels = torch.vstack((kernels,kernels_inv))
print(full_kernels.shape)
weights0 = [rand_weight(k) for k in full_kernels]
weights0 = np.array(weights0)
weights0 = torch.tensor(weights0)
weights0 = weights0.to(torch.float32)
print(weights0.dtype)
print(model2.features[0].weight.shape)
weights0 = nn.Parameter(weights0)
model2.features[0].weight = weights0
model2.to(device)
model2.eval()
print('Test accuracy for substitute = ',test_model(model2,test_loader))

torch.Size([64, 11, 11])
torch.float32
torch.Size([64, 3, 11, 11])
Test accuracy for substitute =  0.011075949367088608
